In [27]:
%load_ext autoreload
%autoreload 2 
import pandas as pd
import numpy as np
import pdfplumber
import re
import nltk
from urllib.parse import urljoin, quote_plus, quote, urlencode
from urllib.request import Request, urlopen
from io import StringIO, BytesIO
from housecredit import HouseCredit
from scraping import DemandDeposit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
 house_abanca = HouseCredit('https://storage.googleapis.com/bank_price_pdfs/3_all_products_210412105930.pdf', [11]).total()

KeyError: None

In [35]:
house_abanca.head(50)

,Commissions,Prices
0,Comissões,NaN
1,NaN,Euros (Mín/Máx)
8,NaN,"250,00"
9,NaN,"500,00"
10,NaN,"275,00 \n130,00"
11,NaN,"10,00 \n50,00"
13,NaN,n/a \nn/a
14,NaN,"(12,00/150,00) \nn/a"
15,NaN,"50,00 \n0,00"
16,NaN,"210,00"


In [24]:
HouseCredit('data/bic.pdf', [11]).total()

,Commissions,Prices
3,NaN,Euros \n(Mín/Máx)
4,Locação Financeira Imobiliária (taxa variável)...,NaN
5,"Habitação própria permanente, secundária ou pa...",NaN
6,Comissões iniciais,NaN
7,1. Comissões contratuais,NaN
8,Montante total \n(valor de aquisição + IMT),"500,00"
9,NaN,"750,00"
10,NaN,--
11,Relocações,"800,00"
12,2. Avaliação de imóveis,NaN


In [10]:
DemandDeposit('https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf', [4]).values()

{'Emissão de extrato': ['Emissão de extrato      1.1  Mensal (enviado ao domicílio)  0,00  nan  IVA – 23%  1.2  Outros, para além do indicado em 1.1  2,00 por extrato  1.3  2ª Via      7,50    2. Fotocópias de segundas vias de talões de depósito   5,00 por documento  nan  IVA – 23%    3.'],
 'Fotocópias de segundas vias de talões de depósito': ['Fotocópias de segundas vias de talões de depósito   5,00 por documento  nan  IVA – 23%    3. Manutenção de conta      3.1 Conta D.O.'],
 'Manutenção de conta': ['Manutenção de conta      3.1 Conta D.O. Particulares (Nota 2)      - Saldo médio até 1.000€  15,00 Trimestral  60,00  - Saldo médio entre 1.001€ e 2.500€  10,00 Trimestral  40,00  IS – 4%    - Saldo médio superior a 2.500€  0,00  nan  3.2 Conta Ordenado (Nota 1 e Nota 16)  0,00  nan  3.3 Conta Standard (não admite novas contratações)  0,00  nan  3.4 Conta Moeda Estrangeira (Nota 12)  5,00 Mensal  60,00  4.'],
 'Levantamento de numerário': ['Levantamento de numerário          7.1 Ao bal

In [25]:
doc = pdfplumber.open('data/bic.pdf')
lista = [11,12]
dataframes = []
for el in lista:
    page = doc.pages[el]
    table = page.extract_table()
    df = pd.DataFrame(table)
    dataframes.append(df)
pd.concat(dataframes)

,0,1,2,3,4,5,6,7,8,9
0,Entrada em vigor: 22-jan-2021\n2. OPERAÇÕES DE...,None,None,None,None,None,None,None,None,None
1,,None,None,None,None,None,None,None,None,None
2,Comissões\nEm %,None,None,None,Comissões,None,None,Acresce \nImposto,Outras \ncondições,None
3,None,None,None,None,Em %,Euros \n(Mín/Máx),Valor \nAnual,None,None,None
4,,Locação Financeira Imobiliária (taxa variável)...,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
14,None,Nota 11 Não aplicável em caso de reembolso ant...,None,None,NaN,NaN,NaN,NaN,NaN,NaN
15,None,Nota 12 Acresce o reembolso das comissões de r...,None,None,NaN,NaN,NaN,NaN,NaN,NaN
16,None,Nota 13 O reembolso antecipado total pode ocor...,None,None,NaN,NaN,NaN,NaN,NaN,NaN
17,,antecedência mínima de 60 (sessenta) dias útei...,None,None,NaN,NaN,NaN,NaN,NaN,NaN
